In [1]:
pip install ultralytics deep-sort-realtime opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.8/993.8 kB 15.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 48.2 MB/s eta 0:00:0000:01:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import numpy as np
import time

# Load YOLOv8 model (trained for multiple objects)
model = YOLO("/kaggle/input/live-model/yolov8n.pt")  # change path if needed

# Init DeepSORT
tracker = DeepSort(max_age=30, n_init=3)

# Loitering setup: track positions & time
loitering_time_threshold = 5  # seconds
track_locations = {}

# Start webcam
cap = cv2.VideoCapture(0)
print("🟢 Surveillance system started. Press 'Q' to stop.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8
    results = model(frame)[0]

    detections = []
    heatmap = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.float32)

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        label = model.names[cls]

        # Add to DeepSORT detections
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, label))
        heatmap[y1:y2, x1:x2] += conf

    # DeepSORT tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    current_time = time.time()

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        label = track.get_det_class()  # e.g., 'person', 'knife', etc.
        x1, y1, x2, y2 = map(int, ltrb)

        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

        # === Loitering Detection (Only for humans) ===
        if label == 'person':
            if track_id not in track_locations:
                track_locations[track_id] = {"pos": (center_x, center_y), "start_time": current_time}
            else:
                prev = track_locations[track_id]
                dist = np.linalg.norm(np.array((center_x, center_y)) - np.array(prev["pos"]))
                if dist < 30:  # same location
                    duration = current_time - prev["start_time"]
                    if duration > loitering_time_threshold:
                        cv2.putText(frame, "🚨 Loitering Detected!", (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
                else:
                    track_locations[track_id] = {"pos": (center_x, center_y), "start_time": current_time}

        # === Object Alert ===
        alert_classes = ['knife', 'gun', 'bag']  # you can customize
        if label in alert_classes:
            cv2.putText(frame, f"⚠ Suspicious Object: {label}", (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)

        # Draw bounding box and ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 100), 2)
        cv2.putText(frame, f"{label} ID:{track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # === Heatmap Overlay ===
    heatmap = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    heatmap_colored = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    final_frame = cv2.addWeighted(frame, 0.7, heatmap_colored, 0.3, 0)

    cv2.imshow("🛡️ Real-Time Surveillance", final_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🟢 Surveillance system started. Press 'Q' to stop.
